In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import bioframe 
import pandas as pd
import numpy as np 
import os
import json
from io import StringIO
import random

import pysam
import h5py

from Bio import motifs
from Bio import pairwise2
from Bio.Seq import Seq

In [2]:
import sys

sys.path.insert(0, "/home1/smaruj/akita_utils/")

# from akita_utils import *
import akita_utils

2022-07-18 16:15:16.742973: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /spack/apps/linux-centos7-x86_64/gcc-8.3.0/python-3.9.2-uvcroioc4witkp6qf7mbebof5ix4wlb6/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/pmix-3.1.3-3sm6emyqaxapunh7rwbjvtaqoqe2e5z3/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/openmpi-4.0.2-ipm3dnvlbtxawpi4ifz7jma6jgr7mexq/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/openblas-0.3.8-2no6mfziiclwxb7lstxoos335gnhjpes/lib:/spack/apps/gcc/8.3.0/lib64::/home1/smaruj/software/GSL/lib:/home1/smaruj/software/HTSLIB/lib
2022-07-18 16:15:16.743007: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
genome_open = pysam.Fastafile("/project/fudenber_735/genomes/mm10/mm10.fa")

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
print("Tensorflow: ", tf.__version__)

from basenji import dataset, seqnn, dna_io, stream

Tensorflow:  2.9.1


In [5]:
num_strong = 200
num_weak = 200

In [6]:
seq_coords_df = akita_utils.prepare_insertion_tsv(
    h5_dirs = '/project/fudenber_735/tensorflow_models/akita/v2/analysis/permute_boundaries_motifs_ctcf_mm10_model*/scd.h5',
    score_key = 'SCD',
    pad_flank = 0, #how much flanking sequence around the sites to include
    weak_thresh_pct = 1, # don't use sites weaker than this, might be artifacts
    weak_num = num_strong,
    strong_thresh_pct = 99, # don't use sites weaker than this, might be artifacts
    strong_num = num_weak ,
    save_tsv=None, # optional filename to save a tsv
)

29 duplicates removed for  /project/fudenber_735/tensorflow_models/akita/v2/analysis/permute_boundaries_motifs_ctcf_mm10_model7/scd.h5
29 duplicates removed for  /project/fudenber_735/tensorflow_models/akita/v2/analysis/permute_boundaries_motifs_ctcf_mm10_model1/scd.h5
annotating each site with boundary-wide scores
filtering sites by overlap with rmsk
df prepared


In [7]:
seq_coords_df

,index,chrom,start,end,strand,genomic_SCD
0,0,chr12,35192359,35192378,-,53.656250
1,1,chr8,102781112,102781131,-,53.500000
2,2,chr5,49961991,49962010,-,53.312500
3,3,chr9,26776356,26776375,-,53.125000
4,4,chr9,51152589,51152608,-,53.062500
...,...,...,...,...,...,...
395,395,chr3,53495939,53495958,+,0.094727
396,396,chr11,77103829,77103848,-,0.095032
397,397,chr8,118855131,118855150,+,0.095093
398,398,chr4,140533539,140533558,+,0.095215


In [12]:
padding = 30

with open("Strong.fasta", "w") as output:
    for i in range(0, num_strong):
        this_row = seq_coords_df.iloc[i]

        if this_row.strand == '+':

            left = Seq(genome_open.fetch(this_row.chrom, this_row.start-padding, this_row.start).upper())
        else:
            left = Seq(genome_open.fetch(this_row.chrom, this_row.end, this_row.end+padding).upper())
            left = left.reverse_complement()
        left = str(left)
        output.write(f"> StrongFlank_{i}")
        output.write("\n")
        output.write(left)
        if i != num_strong-1:
            output.write("\n")

In [13]:
padding = 30

with open("Weak.fasta", "w") as output:
    for i in range(num_strong, num_strong+num_weak):
        this_row = seq_coords_df.iloc[i]

        if this_row.strand == '+':

            left = Seq(genome_open.fetch(this_row.chrom, this_row.start-padding, this_row.start).upper())
        else:
            left = Seq(genome_open.fetch(this_row.chrom, this_row.end, this_row.end+padding).upper())
            left = left.reverse_complement()
        left = str(left)
        output.write(f"> WeakFlank_{i-num_strong}")
        output.write("\n")
        output.write(left)
        if i != num_strong+num_weak-1:
            output.write("\n")